### Data Processor

In [40]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class DataProcessor:
    """
    Class for loading and processing real estate and disaster data
    """
    
    def __init__(self):
        """Initialize the data processor"""
        self.redfin_df = None
        self.fema_df = None
        self.merged_df = None
    
    def load_merged_data(self, filepath):
        """
        Load the pre-merged Redfin and FEMA data from CSV file
        
        Args:
            filepath: Path to the merged data file
            
        Returns:
            DataFrame with processed merged data
        """
        try:
            # Load the data
            df = pd.read_csv(filepath)
            
            # Convert date columns to datetime (without timezone)
            date_cols = ['PERIOD_BEGIN', 'PERIOD_END']
            for col in date_cols:
                if col in df.columns:
                    # First check if timezone info exists and remove it
                    temp_dates = pd.to_datetime(df[col], errors='coerce')
                    if temp_dates.dt.tz is not None:
                        # Convert to naive datetime (no timezone)
                        df[col] = temp_dates.dt.tz_localize(None)
                    else:
                        df[col] = temp_dates
            
            # Add year column if not present
            if 'YEAR' not in df.columns and 'PERIOD_BEGIN' in df.columns:
                df['YEAR'] = df['PERIOD_BEGIN'].dt.year
            
            # Standardize state and city names
            if 'STATE' in df.columns:
                df['STATE'] = df['STATE'].astype(str).str.upper().str.strip()
            
            if 'CITY' in df.columns:
                df['CITY'] = df['CITY'].astype(str).str.title().str.strip()
            
            # Fill missing values for important columns
            numeric_cols = [
                'MEDIAN_SALE_PRICE', 'INVENTORY', 'HOMES_SOLD', 
                'NEW_LISTINGS', 'MEDIAN_DOM', 'AVG_SALE_TO_LIST', 'natural_disaster_score'
            ]
            
            for col in numeric_cols:
                if col in df.columns:
                    # For each column, fill NaNs with median for that city/state group
                    df[col] = df.groupby(['STATE', 'CITY'])[col].transform(
                        lambda x: x.fillna(x.median() if not pd.isna(x.median()) else 0)
                    )
                    
                    # For any remaining NaNs, fill with 0
                    df[col] = df[col].fillna(0)
            
            # Note any data quality issues but don't modify the data
            if 'natural_disaster_score' in df.columns:
                max_disasters = df['natural_disaster_score'].max()
                if max_disasters > 1000:
                    print(f"WARNING: Extremely high disaster counts detected (max: {max_disasters})")
                    print("These values are being preserved in the data but may indicate data quality issues.")
            
            self.merged_df = df
            print(f"Loaded merged data: {df.shape[0]} rows, {df.shape[1]} columns")
            return df
            
        except Exception as e:
            print(f"Error loading merged data: {e}")
            return pd.DataFrame()
    
    def load_fema_data(self, filepath):
        """
        Load FEMA disaster data from CSV file for additional analysis
        
        Args:
            filepath: Path to the FEMA data file
            
        Returns:
            DataFrame with processed FEMA data
        """
        try:
            # Load the data
            df = pd.read_csv(filepath)
            
            # Convert date columns to datetime (without timezone)
            date_cols = ['declarationdate', 'incidentbegindate', 'incidentenddate']
            for col in date_cols:
                if col in df.columns:
                    # First check if timezone info exists and remove it
                    temp_dates = pd.to_datetime(df[col], errors='coerce')
                    if temp_dates.dt.tz is not None:
                        # Convert to naive datetime (no timezone)
                        df[col] = temp_dates.dt.tz_localize(None)
                    else:
                        df[col] = temp_dates
            
            # Standardize text columns
            if 'state' in df.columns:
                df['state'] = df['state'].astype(str).str.upper().str.strip()
                
            if 'incidenttype' in df.columns:
                df['incidenttype'] = df['incidenttype'].astype(str).str.title().str.strip()
            
            # Add year column if not present
            if 'incidentbegindate' in df.columns:
                df['year'] = df['incidentbegindate'].dt.year
            
            self.fema_df = df
            print(f"Loaded FEMA data: {df.shape[0]} rows, {df.shape[1]} columns")
            return df
            
        except Exception as e:
            print(f"Error loading FEMA data: {e}")
            return pd.DataFrame()
    
    def get_location_data(self, state, city):
        """
        Get data for a specific location from the merged dataset
        
        Args:
            state: State code
            city: City name
            
        Returns:
            DataFrame with location-specific data
        """
        if self.merged_df is not None:
            # Filter for location
            location_data = self.merged_df[
                (self.merged_df['STATE'] == state.upper()) &
                (self.merged_df['CITY'] == city.title())
            ]
            
            if location_data.empty:
                print(f"No data found for {city}, {state}")
                return pd.DataFrame()
            
            return location_data
        else:
            print("Error: No merged data loaded")
            return pd.DataFrame()
    
    def get_state_disaster_data(self, state):
        """
        Get disaster data for a specific state from the FEMA dataset
        
        Args:
            state: State code
            
        Returns:
            DataFrame with state-specific disaster data
        """
        if self.fema_df is None:
            print("Error: FEMA data not loaded")
            return pd.DataFrame()
        
        try:
            # Filter for state (no year filtering)
            state_disasters = self.fema_df[self.fema_df['state'] == state.upper()]
            return state_disasters
            
        except Exception as e:
            print(f"Error in get_state_disaster_data: {e}")
            return pd.DataFrame()
    
    def get_state_disaster_summary(self, state):
        """
        Get summary of disasters for the entire state using all available years

        Args:
            state: State code

        Returns:
            Dictionary with disaster metrics
        """
        # First, try to use the FEMA dataset directly (more accurate)
        if self.fema_df is not None:
            fema_state_data = self.get_state_disaster_data(state)

            if not fema_state_data.empty:
                # Count disasters from FEMA data
                total_disasters = len(fema_state_data)

                # Determine years covered
                if 'year' in fema_state_data.columns:
                    years = fema_state_data['year'].unique()
                    years_covered = max(years) - min(years) + 1
                else:
                    years_covered = 14  # Based on your data from 2012-2025

                # Calculate yearly average
                yearly_avg = total_disasters / years_covered

                # Get disaster types
                disaster_types = {}
                if 'incidenttype' in fema_state_data.columns:
                    disaster_types = fema_state_data['incidenttype'].value_counts().to_dict()

                # Determine risk level
                if yearly_avg > 20:
                    risk_level = "high"
                elif yearly_avg > 10:
                    risk_level = "medium"
                else:
                    risk_level = "low"

                return {
                    "total_disasters": total_disasters,
                    "yearly_avg": yearly_avg,
                    "years_covered": years_covered,
                    "disaster_types": disaster_types,
                    "risk_level": risk_level,
                    "source": "FEMA dataset"
                }

        # If FEMA data isn't available or was empty, fall back to merged dataset
        if self.merged_df is not None:
            try:
                # Get state data (without year filtering)
                state_data = self.merged_df[self.merged_df['STATE'] == state.upper()]

                if not state_data.empty:
                    # Calculate years covered from the data or use provided range
                    if 'YEAR' in state_data.columns:
                        years = state_data['YEAR'].unique()
                        years_covered = max(years) - min(years) + 1
                    else:
                        years_covered = 14  # Based on your data from 2012-2025

                    # Count total records for state - this should match your Excel count
                    total_disasters = len(state_data)

                    # Calculate city-specific record counts
                    city_disasters = state_data.groupby('CITY').size().to_dict()

                    # Calculate yearly average for the state
                    yearly_avg = total_disasters / years_covered

                    # Determine risk level for the state
                    if yearly_avg > 20:
                        risk_level = "high"
                    elif yearly_avg > 10:
                        risk_level = "medium"
                    else:
                        risk_level = "low"

                    return {
                        "total_disasters": total_disasters,
                        "yearly_avg": yearly_avg,
                        "years_covered": years_covered,
                        "disaster_types": {},  # No types available from merged data
                        "risk_level": risk_level,
                        "source": "Merged dataset (count of all records)",
                        "city_disasters": city_disasters  # City-specific record counts
                    }
            except Exception as e:
                print(f"Error calculating from merged data: {e}")

        # If all else fails, return default values
        return {
            "total_disasters": 0,
            "yearly_avg": 0,
            "years_covered": 0,
            "disaster_types": {},
            "risk_level": "unknown",
            "source": "No data available"
        }
    
    def get_city_disaster_summary(self, city, state):
        """
        Get summary of disasters for a specific city using all available years

        Args:
            city: City name
            state: State code

        Returns:
            Dictionary with disaster metrics
        """
        try:
            # Get city data (without year filtering)
            city_data = self.merged_df[
                (self.merged_df['STATE'] == state.upper()) &
                (self.merged_df['CITY'] == city.title())
            ]

            if not city_data.empty:
                # Calculate years covered from the data or use provided range
                if 'YEAR' in city_data.columns:
                    years = city_data['YEAR'].unique()
                    years_covered = max(years) - min(years) + 1
                else:
                    years_covered = 14  # Based on your data from 2012-2025

                # Count records as total disasters
                total_disasters = len(city_data)

                # Calculate yearly average for this city
                yearly_avg = total_disasters / years_covered if years_covered > 0 else 0

                # Determine risk level for this city
                if yearly_avg > 20:
                    risk_level = "high"
                elif yearly_avg > 10:
                    risk_level = "medium"
                else:
                    risk_level = "low"

                return {
                    "total_disasters": total_disasters,
                    "yearly_avg": yearly_avg,
                    "years_covered": years_covered,
                    "risk_level": risk_level,
                    "source": "Merged dataset (city-specific record count)"
                }
        except Exception as e:
            print(f"Error calculating city disaster summary: {e}")

        # If any errors or no data, return default values
        return {
            "total_disasters": 0,
            "yearly_avg": 0,
            "years_covered": 0,
            "risk_level": "unknown",
            "source": "No data available for this city"
        }
    
    def get_available_states(self):
        """Get list of available states in the data"""
        if self.merged_df is not None and 'STATE' in self.merged_df.columns:
            # Group by STATE to find states with data
            state_counts = self.merged_df.groupby('STATE').size()
            states = state_counts.index.tolist()
            return sorted(states)
        return []
    
    def get_available_cities(self, state):
        """
        Get list of all available cities for a state
        
        Args:
            state: State code
            
        Returns:
            List of city names
        """
        if self.merged_df is not None and 'CITY' in self.merged_df.columns and 'STATE' in self.merged_df.columns:
            # Group by CITY within this state to find cities with data
            state_data = self.merged_df[self.merged_df['STATE'] == state.upper()]
            city_counts = state_data.groupby('CITY').size()
            cities = city_counts.index.tolist()
            city_count = len(cities)
            
            # Print information about city counts for transparency
            if city_count > 500:
                print(f"Note: {state} has {city_count} unique cities in dataset.")
                print("This is more than the expected number of municipalities and may include")
                print("neighborhoods, misspellings, or other data quality issues.")
                
            return sorted(cities)
        return []
    
    def get_city_stats(self, state):
        """
        Get detailed statistics about cities in a state

        Args:
            state: State code

        Returns:
            Dictionary with city statistics
        """
        stats = {
            "total_cities": 0,
            "cities_with_1_record": 0,
            "cities_with_few_records": 0,
            "top_cities_by_records": []
        }

        if self.merged_df is not None and 'CITY' in self.merged_df.columns and 'STATE' in self.merged_df.columns:
            # Filter data to only include records from this state
            state_data = self.merged_df[self.merged_df['STATE'] == state.upper()]

            # Get unique cities in this state
            unique_cities = state_data['CITY'].unique()

            # Count records per city
            city_counts = state_data.groupby('CITY').size()

            stats["total_cities"] = len(unique_cities)  # Number of unique cities
            stats["cities_with_1_record"] = sum(city_counts == 1)
            stats["cities_with_few_records"] = sum(city_counts < 5)

            # Get top cities by record count
            top_cities = city_counts.sort_values(ascending=False).head(10)
            stats["top_cities_by_records"] = [{"city": city, "records": count} for city, count in top_cities.items()]

        return stats
    
    def calculate_city_price_metrics(self, city, state):
        """
        Calculate price metrics for a specific city
        
        Args:
            city: City name
            state: State code
            
        Returns:
            Dictionary with price metrics
        """
        metrics = {
            "avg_price": 0,
            "median_price": 0,
            "min_price": 0,
            "max_price": 0,
            "price_volatility": 0,
            "record_count": 0,
            "years_covered": 0
        }
        
        try:
            # Get location data for this specific city
            location_data = self.get_location_data(state, city)
            
            if location_data.empty or 'MEDIAN_SALE_PRICE' not in location_data.columns:
                return metrics
            
            # Calculate years covered from the data
            if 'YEAR' in location_data.columns:
                years = location_data['YEAR'].unique()
                metrics["years_covered"] = max(years) - min(years) + 1
            
            # Calculate price metrics for this specific city
            prices = location_data['MEDIAN_SALE_PRICE'].dropna()
            
            metrics["avg_price"] = prices.mean()
            metrics["median_price"] = prices.median()
            metrics["min_price"] = prices.min()
            metrics["max_price"] = prices.max()
            metrics["record_count"] = len(location_data)
            
            # Calculate price volatility
            location_data = location_data.sort_values('PERIOD_BEGIN')
            price_changes = location_data['MEDIAN_SALE_PRICE'].pct_change().dropna()
            
            if len(price_changes) >= 2:
                metrics["price_volatility"] = price_changes.std()
            
            return metrics
            
        except Exception as e:
            print(f"Error calculating city price metrics: {e}")
            return metrics
    
    def calculate_price_volatility(self, city, state):
        """
        Calculate price volatility for a specific city
        
        Args:
            city: City name
            state: State code
            
        Returns:
            Volatility score (standard deviation of price changes)
        """
        try:
            # This should only use data for the specific city
            location_data = self.get_location_data(state, city)
            
            if location_data.empty or 'MEDIAN_SALE_PRICE' not in location_data.columns:
                return 0.05  # Default value
            
            # Sort by date
            location_data = location_data.sort_values('PERIOD_BEGIN')
            
            # Calculate price changes
            price_changes = location_data['MEDIAN_SALE_PRICE'].pct_change().dropna()
            
            if len(price_changes) < 2:
                return 0.05  # Default value
            
            # Return standard deviation as volatility measure
            return price_changes.std()
            
        except Exception as e:
            print(f"Error calculating price volatility: {e}")
            return 0.05  # Default value
    
    def get_data_quality_report(self):
        """
        Generate a data quality report for the loaded data
        
        Returns:
            Dictionary with data quality metrics
        """
        report = {
            "merged_data": {
                "rows": 0,
                "columns": 0,
                "missing_values": {},
                "suspicious_values": {}
            },
            "fema_data": {
                "rows": 0,
                "columns": 0
            }
        }
        
        # Check merged data
        if self.merged_df is not None:
            report["merged_data"]["rows"] = len(self.merged_df)
            report["merged_data"]["columns"] = len(self.merged_df.columns)
            
            # Check for missing values
            missing = self.merged_df.isnull().sum()
            missing = missing[missing > 0]
            report["merged_data"]["missing_values"] = missing.to_dict()
            
            # Check for suspicious disaster counts
            if 'natural_disaster_score' in self.merged_df.columns:
                suspicious = self.merged_df[self.merged_df['natural_disaster_score'] > 100]
                if not suspicious.empty:
                    report["merged_data"]["suspicious_values"]["high_disaster_count"] = len(suspicious)
        
        # Check FEMA data
        if self.fema_df is not None:
            report["fema_data"]["rows"] = len(self.fema_df)
            report["fema_data"]["columns"] = len(self.fema_df.columns)
            
        return report


# Example usage
if __name__ == "__main__":
    # Create data processor instance
    processor = DataProcessor()
    
    # Load the merged dataset that contains both Redfin and FEMA data
    merged_data = processor.load_merged_data("/Users/shivangi/Downloads/redfin_with_disaster_score.csv")
    
    # Optionally load separate FEMA data for more detailed disaster analysis
    fema_data = processor.load_fema_data("/Users/shivangi/Downloads/fema_cleaned.csv")
    
    # Get data quality report
    quality_report = processor.get_data_quality_report()
    print("\nData Quality Report:")
    print(f"Merged data: {quality_report['merged_data']['rows']} rows, {quality_report['merged_data']['columns']} columns")
    if quality_report['merged_data']['suspicious_values']:
        print("Found suspicious values:")
        for key, value in quality_report['merged_data']['suspicious_values'].items():
            print(f"- {key}: {value} records")
    
    # Get available states and print the count
    available_states = processor.get_available_states()
    print(f"\nFound {len(available_states)} available states")
    
    if available_states:
        # Print first few states
        print(f"Sample states: {', '.join(available_states[:10])}")
        
        # Test with the first available state
        test_state = available_states[7]
        available_cities = processor.get_available_cities(test_state)
        
        # Get city stats
        city_stats = processor.get_city_stats(test_state)
        print(f"\nState {test_state} has {city_stats['total_cities']} cities")
        print(f"Cities with only 1 record: {city_stats['cities_with_1_record']}")
        print(f"Cities with fewer than 5 records: {city_stats['cities_with_few_records']}")
        
        print(f"\nTop cities by record count:")
        for city_data in city_stats['top_cities_by_records'][:5]:
            print(f"- {city_data['city']}: {city_data['records']} records")
        
        if available_cities:
            # Print first few cities
            print(f"\nSample cities: {', '.join(available_cities[:5])}")
            
            # Test with the first available city
            test_city = available_cities[3]
            print(f"\nTesting with {test_city}, {test_state}")
            
            # Get location data
            location_data = processor.get_location_data(test_state, test_city)
            if not location_data.empty:
                # Print basic stats
                print(f"Found {len(location_data)} records")
                
                # Calculate city-specific price metrics
                price_metrics = processor.calculate_city_price_metrics(test_city, test_state)
                print(f"Average price: ${price_metrics['avg_price']:.2f}")
                print(f"Price volatility: {price_metrics['price_volatility']:.4f}")
                print(f"Years covered: {price_metrics['years_covered']}")
            
            # Get disaster summary for the state
            disaster_summary = processor.get_state_disaster_summary(test_state)
            print(f"\nDisaster summary for {test_state}:")
            print(f"Total disasters: {disaster_summary['total_disasters']}")
            print(f"Yearly average: {disaster_summary['yearly_avg']:.2f}")
            print(f"Years covered: {disaster_summary['years_covered']}")
            print(f"Risk level: {disaster_summary['risk_level']}")
            print(f"Source: {disaster_summary['source']}")
            
            # Print disaster types if available
            if disaster_summary['disaster_types']:
                print("\nTop disaster types:")
                top_disasters = sorted(disaster_summary['disaster_types'].items(), key=lambda x: x[1], reverse=True)[:5]
                for disaster_type, count in top_disasters:
                    print(f"- {disaster_type}: {count}")
            
            if 'note' in disaster_summary:
                print(f"\nNote: {disaster_summary['note']}")
            
            # Get city-specific disaster summary (new functionality)
            city_disaster_summary = processor.get_city_disaster_summary(test_city, test_state)
            print(f"\nDisaster summary for {test_city}, {test_state}:")
            print(f"Total disasters: {city_disaster_summary['total_disasters']}")
            print(f"Yearly average: {city_disaster_summary['yearly_avg']:.2f}")
            print(f"Years covered: {city_disaster_summary['years_covered']}")
            print(f"Risk level: {city_disaster_summary['risk_level']}")
            print(f"Source: {city_disaster_summary['source']}")
            
            # If the state summary has city-specific disaster counts, show this city's contribution
            if 'city_disasters' in disaster_summary and test_city.title() in disaster_summary['city_disasters']:
                city_count = disaster_summary['city_disasters'][test_city.title()]
                print(f"\nThis city's contribution to state total: {city_count} disasters")

Loaded merged data: 5812149 rows, 58 columns
Loaded FEMA data: 20000 rows, 9 columns

Data Quality Report:
Merged data: 5812149 rows, 58 columns

Found 51 available states
Sample states: ALABAMA, ALASKA, ARIZONA, ARKANSAS, CALIFORNIA, COLORADO, COLUMBIA, CONNECTICUT, DELAWARE, FLORIDA

State CONNECTICUT has 215 cities
Cities with only 1 record: 0
Cities with fewer than 5 records: 1

Top cities by record count:
- Middletown: 671 records
- Hartford: 637 records
- Bristol: 636 records
- Norwich: 636 records
- Manchester: 635 records

Sample cities: Ansonia, Ball Pond, Baltic, Bantam, Bethel

Testing with Bantam, CONNECTICUT
Found 205 records
Average price: $297616.78
Price volatility: 1.1748
Years covered: 14

Disaster summary for CONNECTICUT:
Total disasters: 69155
Yearly average: 4939.64
Years covered: 14
Risk level: high
Source: Merged dataset (count of all records)

Disaster summary for Bantam, CONNECTICUT:
Total disasters: 205
Yearly average: 14.64
Years covered: 14
Risk level: mediu

In [44]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

class HourlyWeatherAnalyzer:
    """
    Class for retrieving and analyzing hourly weather data from NOAA
    to assess climate-related risks for real estate investments.
    """
    
    def __init__(self, noaa_token=None):
        """
        Initialize the weather analyzer
        
        Args:
            noaa_token: API token for NOAA data access
        """
        self.noaa_token = noaa_token
        self.cache_dir = "weather_cache"
        
        # Create cache directory if it doesn't exist
        if not os.path.exists(self.cache_dir):
            os.makedirs(self.cache_dir)
    
    def set_api_token(self, token):
        """
        Set the NOAA API token
        
        Args:
            token: API token
        """
        self.noaa_token = token
    
    def get_hourly_weather_data(self, city, state):
        """
        Get hourly weather data for a city for the past year
        
        Args:
            city: City name
            state: State code
            
        Returns:
            DataFrame with hourly weather data or dictionary with aggregated metrics if API fails
        """
        try:
            # Check cache first
            cache_file = os.path.join(self.cache_dir, f"{state.upper()}_{city.title()}_hourly.csv")
            if os.path.exists(cache_file):
                # Check if cache is fresh (less than 1 day old)
                cache_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
                if (datetime.now() - cache_time).days < 1:
                    print(f"Using cached hourly weather data for {city}, {state}")
                    return pd.read_csv(cache_file)
            
            # Set date range for the past year
            end_date = datetime.now()
            start_date = end_date - timedelta(days=365)
            
            # Format dates for API
            start_str = start_date.strftime('%Y-%m-%dT00:00:00')
            end_str = end_date.strftime('%Y-%m-%dT23:59:59')
            
            if not self.noaa_token:
                print("No NOAA API token provided. Using generated sample data.")
                # Generate sample hourly data
                return self._generate_sample_hourly_data(city, state, start_date, end_date)
            
            print(f"Fetching hourly weather data for {city}, {state}...")
            
            # Prepare API request for hourly data
            headers = {"token": self.noaa_token}
            base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
            
            # Use GHCND (Global Historical Climatology Network - Daily) dataset
            # with hourly precipitation, temperature, and other elements
            params = {
                "datasetid": "NORMAL_HLY",  # Hourly Normals dataset
                "locationid": f"CITY:{city.upper()}",
                "startdate": start_str,
                "enddate": end_str,
                "units": "standard",
                "limit": 1000
            }
            
            # Make API request
            response = requests.get(base_url, headers=headers, params=params)
            
            if response.status_code != 200:
                print(f"Error fetching hourly data: {response.status_code}")
                print("Falling back to GHCND dataset with daily data...")
                
                # Fall back to daily data (GHCND dataset)
                params = {
                    "datasetid": "GHCND",
                    "locationid": f"CITY:{city.upper()}",
                    "startdate": start_date.strftime('%Y-%m-%d'),
                    "enddate": end_date.strftime('%Y-%m-%d'),
                    "units": "standard",
                    "limit": 1000
                }
                
                response = requests.get(base_url, headers=headers, params=params)
                
                if response.status_code != 200:
                    print(f"Error fetching daily data: {response.status_code}")
                    print("Using generated sample data instead.")
                    # Generate sample data as fallback
                    return self._generate_sample_hourly_data(city, state, start_date, end_date)
            
            # Parse response
            data = response.json()
            
            if 'results' not in data or not data['results']:
                print("No data returned from API. Using generated sample data.")
                return self._generate_sample_hourly_data(city, state, start_date, end_date)
            
            # Convert to DataFrame
            df = pd.DataFrame(data['results'])
            
            # Save to cache
            df.to_csv(cache_file, index=False)
            
            return df
            
        except Exception as e:
            print(f"Error retrieving weather data: {e}")
            print("Using generated sample data instead.")
            # Generate sample data as fallback
            return self._generate_sample_hourly_data(city, state, start_date, end_date)
    
    def _generate_sample_hourly_data(self, city, state, start_date, end_date):
        """
        Generate sample hourly weather data for testing
        
        Args:
            city: City name
            state: State code
            start_date: Start date
            end_date: End date
            
        Returns:
            DataFrame with sample hourly data
        """
        print(f"Generating sample hourly weather data for {city}, {state}...")
        
        # Generate hourly timestamps for the entire year
        hours = int((end_date - start_date).total_seconds() / 3600) + 1
        timestamps = [start_date + timedelta(hours=i) for i in range(hours)]
        
        # Base parameters that vary by location (rough estimates)
        location_params = {
            'TX': {'temp_mean': 75, 'temp_amplitude': 20, 'precip_prob': 0.05, 'humid_mean': 60},
            'FL': {'temp_mean': 80, 'temp_amplitude': 15, 'precip_prob': 0.15, 'humid_mean': 75},
            'NY': {'temp_mean': 55, 'temp_amplitude': 25, 'precip_prob': 0.10, 'humid_mean': 65},
            'CA': {'temp_mean': 65, 'temp_amplitude': 15, 'precip_prob': 0.05, 'humid_mean': 50},
            'WA': {'temp_mean': 50, 'temp_amplitude': 15, 'precip_prob': 0.20, 'humid_mean': 70}
        }
        
        # Use the state params or default if not found
        params = location_params.get(state.upper(), 
                                     {'temp_mean': 65, 'temp_amplitude': 20, 'precip_prob': 0.10, 'humid_mean': 60})
        
        # Create DataFrame
        df = pd.DataFrame({
            'date': timestamps,
            'city': city,
            'state': state
        })
        
        # Add hour and month for seasonal patterns
        df['hour'] = df['date'].dt.hour
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
        
        # Generate temperature (°F) with daily and seasonal variations
        # Daily cycle: lowest at ~5 AM, highest at ~3 PM
        daily_cycle = np.sin(2 * np.pi * (df['hour'] - 5) / 24)
        # Seasonal cycle: lowest in January, highest in July
        seasonal_cycle = np.sin(2 * np.pi * (df['month'] - 1) / 12)
        
        # Combine cycles with random noise
        temp_base = params['temp_mean']
        daily_amplitude = 15  # Daily temperature swing
        seasonal_amplitude = params['temp_amplitude']
        random_noise = np.random.normal(0, 5, len(df))  # Random fluctuations
        
        df['temperature'] = (
            temp_base +
            daily_amplitude * daily_cycle +
            seasonal_amplitude * seasonal_cycle +
            random_noise
        )
        
        # Generate humidity (%) with inverse relationship to temperature
        humid_base = params['humid_mean']
        humid_daily = -0.5 * daily_cycle  # Humidity tends to be lower when temperature is higher
        humid_seasonal = -0.3 * seasonal_cycle
        humid_noise = np.random.normal(0, 10, len(df))
        
        df['humidity'] = np.clip(
            humid_base +
            10 * humid_daily +
            15 * humid_seasonal +
            humid_noise,
            10, 100  # Clip to valid humidity range
        )
        
        # Generate precipitation (inches)
        # Higher probability in certain months and when humidity is high
        rain_prob = params['precip_prob'] * (1 + 0.5 * seasonal_cycle) * (df['humidity'] / 50)
        is_raining = np.random.random(len(df)) < rain_prob
        
        # Amount when raining follows exponential distribution
        rain_amount = np.zeros(len(df))
        rain_amount[is_raining] = np.random.exponential(0.1, size=is_raining.sum())
        
        df['precipitation'] = rain_amount
        
        # Generate wind speed (mph)
        wind_base = 5 + 5 * abs(seasonal_cycle)  # More wind in winter and summer
        wind_noise = np.random.exponential(3, len(df))
        
        df['wind_speed'] = wind_base + wind_noise
        
        # Calculate "feels like" temperature based on wind chill and heat index
        df['feels_like'] = df['temperature'].copy()
        
        # Apply wind chill adjustment when cold
        cold_mask = df['temperature'] < 50
        df.loc[cold_mask, 'feels_like'] = df.loc[cold_mask, 'temperature'] - 0.7 * df.loc[cold_mask, 'wind_speed']
        
        # Apply heat index adjustment when hot and humid
        hot_mask = (df['temperature'] > 80) & (df['humidity'] > 40)
        heat_adjustment = 0.1 * (df.loc[hot_mask, 'temperature'] - 80) * (df.loc[hot_mask, 'humidity'] / 100)
        df.loc[hot_mask, 'feels_like'] += heat_adjustment
        
        # Save to cache
        cache_file = os.path.join(self.cache_dir, f"{state.upper()}_{city.title()}_hourly.csv")
        df.to_csv(cache_file, index=False)
        
        print(f"Generated {len(df)} hours of sample weather data")
        return df
    
    def analyze_weather_data(self, city, state):
        """
        Analyze hourly weather data to calculate climate metrics
        
        Args:
            city: City name
            state: State code
            
        Returns:
            Dictionary with weather analysis and risk metrics
        """
        # Get hourly data
        df = self.get_hourly_weather_data(city, state)
        
        # Check if we received a DataFrame or dict
        if isinstance(df, dict):
            # We got aggregated metrics directly
            metrics = df
        else:
            # Calculate metrics from the DataFrame
            metrics = self._calculate_metrics_from_dataframe(df, city, state)
        
        # Calculate risk score
        risk_score, risk_level, risk_factors, positive_factors = self._calculate_risk_score(metrics)
        
        # Combine everything into analysis result
        analysis = {
            'city': city,
            'state': state,
            'metrics': metrics,
            'risk_score': risk_score,
            'risk_level': risk_level,
            'risk_factors': risk_factors,
            'positive_factors': positive_factors,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        return analysis
    
    def _calculate_metrics_from_dataframe(self, df, city, state):
        """
        Calculate weather metrics from hourly data
        
        Args:
            df: DataFrame with hourly weather data
            city: City name
            state: State code
            
        Returns:
            Dictionary with weather metrics
        """
        metrics = {
            'city': city,
            'state': state,
            'data_points': len(df)
        }
        
        # Temperature metrics
        if 'temperature' in df.columns:
            metrics['temp_avg'] = df['temperature'].mean()
            metrics['temp_max'] = df['temperature'].max()
            metrics['temp_min'] = df['temperature'].min()
            metrics['temp_range'] = metrics['temp_max'] - metrics['temp_min']
            
            # Temperature extremes
            metrics['days_over_90'] = len(df[df['temperature'] > 90]) / 24  # Convert hours to days
            metrics['days_over_100'] = len(df[df['temperature'] > 100]) / 24
            metrics['days_below_32'] = len(df[df['temperature'] < 32]) / 24
            metrics['days_below_20'] = len(df[df['temperature'] < 20]) / 24
            
            # Temperature variability (standard deviation of daily highs)
            if 'date' in df.columns:
                daily_highs = df.groupby(df['date'].dt.date)['temperature'].max()
                metrics['temp_variability'] = daily_highs.std()
        
        # Humidity metrics
        if 'humidity' in df.columns:
            metrics['humidity_avg'] = df['humidity'].mean()
            metrics['humidity_max'] = df['humidity'].max()
            metrics['humidity_min'] = df['humidity'].min()
            
            # Extreme humidity
            metrics['hours_high_humidity'] = len(df[df['humidity'] > 80])
            metrics['percent_high_humidity'] = metrics['hours_high_humidity'] / len(df) * 100
        
        # Precipitation metrics
        if 'precipitation' in df.columns:
            metrics['precip_total'] = df['precipitation'].sum()
            metrics['precip_max_hourly'] = df['precipitation'].max()
            
            # Rain days and heavy rain
            rain_hours = df[df['precipitation'] > 0]
            if 'date' in df.columns:
                rain_days = rain_hours['date'].dt.date.nunique()
            else:
                # Approximate if date format isn't compatible
                rain_days = len(rain_hours) / 24  # Rough estimate
            
            metrics['rain_days'] = rain_days
            metrics['heavy_rain_hours'] = len(df[df['precipitation'] > 0.3])  # Hours with >0.3" rain
        
        # Wind metrics
        if 'wind_speed' in df.columns:
            metrics['wind_avg'] = df['wind_speed'].mean()
            metrics['wind_max'] = df['wind_speed'].max()
            metrics['hours_high_wind'] = len(df[df['wind_speed'] > 20])
        
        # Comfort metrics
        if 'temperature' in df.columns and 'humidity' in df.columns:
            # Comfortable conditions: 65-85°F with 30-60% humidity
            comfortable = (
                (df['temperature'] >= 65) & 
                (df['temperature'] <= 85) & 
                (df['humidity'] >= 30) & 
                (df['humidity'] <= 60)
            )
            metrics['comfortable_hours'] = len(df[comfortable])
            metrics['percent_comfortable'] = metrics['comfortable_hours'] / len(df) * 100
            
            # Heat index for hours where it applies
            hot_humid = (df['temperature'] >= 80) & (df['humidity'] >= 40)
            if 'feels_like' in df.columns:
                heat_index_diff = df.loc[hot_humid, 'feels_like'] - df.loc[hot_humid, 'temperature']
                metrics['avg_heat_index_effect'] = heat_index_diff.mean() if not heat_index_diff.empty else 0
        
        # Seasonal analysis
        if 'date' in df.columns and 'temperature' in df.columns:
            # Group by season
            df['month'] = df['date'].dt.month
            df['season'] = pd.cut(
                df['month'],
                bins=[0, 3, 6, 9, 12],
                labels=['Winter', 'Spring', 'Summer', 'Fall'],
                include_lowest=True
            )
            
            # Calculate seasonal averages
            season_temps = df.groupby('season')['temperature'].mean().to_dict()
            metrics['seasonal_temps'] = season_temps
            
            if 'precipitation' in df.columns:
                season_precip = df.groupby('season')['precipitation'].sum().to_dict()
                metrics['seasonal_precip'] = season_precip
        
        return metrics
    
    def _calculate_risk_score(self, metrics):
        """
        Calculate weather risk score based on metrics
        
        Args:
            metrics: Dictionary with weather metrics
            
        Returns:
            tuple of (risk_score, risk_level, risk_factors, positive_factors)
        """
        risk_score = 0  # 0-100 scale
        risk_factors = []
        positive_factors = []
        
        # Temperature extremes
        if 'days_over_90' in metrics:
            hot_days = metrics['days_over_90']
            if hot_days > 90:  # More than 3 months of 90°+ weather
                risk_score += 15
                risk_factors.append(f"Extreme heat: {hot_days:.1f} days above 90°F per year")
            elif hot_days > 45:  # More than 1.5 months of 90°+ weather
                risk_score += 10
                risk_factors.append(f"High heat: {hot_days:.1f} days above 90°F per year")
            elif hot_days < 10:
                positive_factors.append("Minimal extreme heat days")
        
        if 'days_below_32' in metrics:
            cold_days = metrics['days_below_32']
            if cold_days > 90:  # More than 3 months of freezing weather
                risk_score += 15
                risk_factors.append(f"Extreme cold: {cold_days:.1f} freezing days per year")
            elif cold_days > 45:
                risk_score += 10
                risk_factors.append(f"Cold climate: {cold_days:.1f} freezing days per year")
            elif cold_days < 10:
                positive_factors.append("Minimal freezing days")
        
        # Temperature variability
        if 'temp_variability' in metrics:
            variability = metrics['temp_variability']
            if variability > 15:
                risk_score += 10
                risk_factors.append(f"High temperature variability: {variability:.1f}°F std dev")
            elif variability < 8:
                positive_factors.append("Stable temperature patterns")
        
        # Precipitation
        if 'precip_total' in metrics:
            annual_precip = metrics['precip_total']
            if annual_precip > 60:
                risk_score += 15
                risk_factors.append(f"Heavy precipitation: {annual_precip:.1f} inches annually")
            elif annual_precip > 45:
                risk_score += 10
                risk_factors.append(f"High precipitation: {annual_precip:.1f} inches annually")
            elif annual_precip < 15:
                risk_score += 12
                risk_factors.append(f"Drought-prone: Only {annual_precip:.1f} inches annually")
            elif 20 <= annual_precip <= 40:
                positive_factors.append(f"Moderate precipitation: {annual_precip:.1f} inches annually")
        
        # Heavy rain events
        if 'heavy_rain_hours' in metrics:
            heavy_rain = metrics['heavy_rain_hours']
            if heavy_rain > 50:
                risk_score += 15
                risk_factors.append(f"Flood risk: {heavy_rain} hours of heavy rain annually")
            elif heavy_rain > 25:
                risk_score += 10
                risk_factors.append(f"Heavy rainfall events: {heavy_rain} hours annually")
        
        # Humidity
        if 'percent_high_humidity' in metrics:
            high_humidity = metrics['percent_high_humidity']
            if high_humidity > 40:
                risk_score += 10
                risk_factors.append(f"High humidity: {high_humidity:.1f}% of hours above 80% humidity")
            elif high_humidity < 15:
                positive_factors.append("Low humidity levels")
        
        # Wind
        if 'hours_high_wind' in metrics:
            high_wind = metrics['hours_high_wind']
            if high_wind > 500:  # More than 20 days worth of high wind
                risk_score += 12
                risk_factors.append(f"High wind exposure: {high_wind} hours of strong winds annually")
            elif high_wind > 200:
                risk_score += 8
                risk_factors.append(f"Moderate wind exposure: {high_wind} hours annually")
        
        # Comfort conditions
        if 'percent_comfortable' in metrics:
            comfort = metrics['percent_comfortable']
            if comfort > 60:
                positive_factors.append(f"Excellent comfort: {comfort:.1f}% of hours have ideal conditions")
                risk_score -= 10  # Reduce risk score for very comfortable climates
            elif comfort > 40:
                positive_factors.append(f"Good comfort: {comfort:.1f}% of hours have ideal conditions")
                risk_score -= 5
            elif comfort < 20:
                risk_score += 8
                risk_factors.append(f"Poor comfort: Only {comfort:.1f}% of hours have ideal conditions")
        
        # Heat index effect
        if 'avg_heat_index_effect' in metrics:
            heat_effect = metrics['avg_heat_index_effect']
            if heat_effect > 8:
                risk_score += 10
                risk_factors.append(f"Dangerous heat index effect: +{heat_effect:.1f}°F average increase")
            elif heat_effect > 5:
                risk_score += 5
                risk_factors.append(f"Significant heat index effect: +{heat_effect:.1f}°F average increase")
        
        # Seasonal extremes
        if 'seasonal_temps' in metrics:
            season_temps = metrics['seasonal_temps']
            max_season = max(season_temps.items(), key=lambda x: x[1])
            min_season = min(season_temps.items(), key=lambda x: x[1])
            seasonal_range = max_season[1] - min_season[1]
            
            if seasonal_range > 40:
                risk_score += 10
                risk_factors.append(f"Extreme seasonal variation: {seasonal_range:.1f}°F difference between {min_season[0]} and {max_season[0]}")
            elif seasonal_range < 20:
                positive_factors.append(f"Mild seasonal variation: Only {seasonal_range:.1f}°F difference between seasons")
        
        # Determine risk level
        if risk_score >= 50:
            risk_level = 'high'
        elif risk_score >= 30:
            risk_level = 'medium'
        else:
            risk_level = 'low'
        
        return risk_score, risk_level, risk_factors, positive_factors
    
    def generate_weather_summary(self, city, state):
        """
        Generate a human-readable summary of weather conditions and risks
        
        Args:
            city: City name
            state: State code
            
        Returns:
            String with formatted weather summary
        """
        # Get analysis
        analysis = self.analyze_weather_data(city, state)
        metrics = analysis['metrics']
        
        # Format summary
        summary = f"WEATHER ANALYSIS FOR {city.upper()}, {state.upper()}\n"
        summary += "=" * 50 + "\n\n"
        
        # Temperature
        summary += "TEMPERATURE PROFILE:\n"
        if 'temp_avg' in metrics:
            summary += f"• Average Temperature: {metrics['temp_avg']:.1f}°F\n"
        if 'temp_max' in metrics and 'temp_min' in metrics:
            summary += f"• Temperature Range: {metrics['temp_min']:.1f}°F to {metrics['temp_max']:.1f}°F\n"
        if 'days_over_90' in metrics:
            summary += f"• Hot Days: {metrics['days_over_90']:.1f} days above 90°F per year\n"
        if 'days_below_32' in metrics:
            summary += f"• Freezing Days: {metrics['days_below_32']:.1f} days below 32°F per year\n"
        
        # Precipitation
        summary += "\nPRECIPITATION PROFILE:\n"
        if 'precip_total' in metrics:
            summary += f"• Annual Precipitation: {metrics['precip_total']:.1f} inches\n"
        if 'rain_days' in metrics:
            summary += f"• Rain Days: {metrics['rain_days']:.1f} days per year\n"
        if 'heavy_rain_hours' in metrics:
            summary += f"• Heavy Rain: {metrics['heavy_rain_hours']} hours of heavy rainfall per year\n"
        
        # Humidity
        summary += "\nHUMIDITY PROFILE:\n"
        if 'humidity_avg' in metrics:
            summary += f"• Average Humidity: {metrics['humidity_avg']:.1f}%\n"
        if 'percent_high_humidity' in metrics:
            summary += f"• High Humidity: {metrics['percent_high_humidity']:.1f}% of hours above 80% humidity\n"
        
        # Comfort metrics
        summary += "\nCOMFORT METRICS:\n"
        if 'percent_comfortable' in metrics:
            summary += f"• Comfortable Conditions: {metrics['percent_comfortable']:.1f}% of hours\n"
        if 'avg_heat_index_effect' in metrics:
            summary += f"• Heat Index Effect: +{metrics['avg_heat_index_effect']:.1f}°F (when applicable)\n"
        
        # Season breakdown
        if 'seasonal_temps' in metrics:
            summary += "\nSEASONAL BREAKDOWN:\n"
            for season, temp in metrics['seasonal_temps'].items():
                precip = metrics.get('seasonal_precip', {}).get(season, 0)
                summary += f"• {season}: {temp:.1f}°F avg, {precip:.1f} inches of precipitation\n"
        
        # Risk assessment
        summary += f"\nWEATHER RISK ASSESSMENT: {analysis['risk_level'].upper()}\n"
        summary += f"Risk Score: {analysis['risk_score']:.1f}/100\n\n"
        
        if analysis['risk_factors']:
            summary += "Risk Factors:\n"
            for factor in analysis['risk_factors']:
                summary += f"• {factor}\n"
            summary += "\n"
        
        if analysis['positive_factors']:
            summary += "Positive Factors:\n"
            for factor in analysis['positive_factors']:
                summary += f"• {factor}\n"
            summary += "\n"
        
        # Investment implications
        summary += "INVESTMENT IMPLICATIONS:\n"
        if analysis['risk_level'] == 'high':
            summary += "This location has significant weather-related risks that could impact property values, "
            summary += "maintenance costs, and insurance rates. Consider these factors carefully in your investment decision."
        elif analysis['risk_level'] == 'medium':
            summary += "This location has moderate weather-related risks to consider. Some seasonal challenges "
            summary += "may require additional maintenance or preparation, but they are generally manageable."
        else:
            summary += "This location has favorable weather conditions for real estate investment. "
            summary += "The climate presents minimal challenges for property maintenance and livability."
        
        return summary
    
    def plot_weather_risk(self, city, state):
        """
        Create a visualization of weather risk analysis
        
        Args:
            city: City name
            state: State code
            
        Returns:
            Matplotlib figure
        """
        # Get analysis
        analysis = self.analyze_weather_data(city, state)
        metrics = analysis['metrics']
        
        # Create figure with subplots
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        fig.suptitle(f"Weather Risk Analysis for {city}, {state}", fontsize=16)
        
        # Temperature plot (top left)
        ax1 = axes[0, 0]
        if all(key in metrics for key in ['temp_min', 'temp_avg', 'temp_max']):
            temp_data = [metrics['temp_min'], metrics['temp_avg'], metrics['temp_max']]
            temp_labels = ['Min', 'Avg', 'Max']
            
            ax1.bar(temp_labels, temp_data, color=['blue', 'green', 'red'])
            ax1.set_title('Temperature Profile (°F)')
            ax1.set_ylim(min(0, metrics['temp_min'] - 10), metrics['temp_max'] + 10)
            
            # Add extreme day counts
            if 'days_over_90' in metrics and 'days_below_32' in metrics:
                ax1.text(0.05, 0.05, 
                         f"Days >90°F: {metrics['days_over_90']:.1f}\nDays <32°F: {metrics['days_below_32']:.1f}",
                         transform=ax1.transAxes, bbox=dict(facecolor='white', alpha=0.8))
        
        # Precipitation plot (top right)
        ax2 = axes[0, 1]
        if 'seasonal_precip' in metrics:
            seasons = list(metrics['seasonal_precip'].keys())
            precip_data = list(metrics['seasonal_precip'].values())
            
            ax2.bar(seasons, precip_data, color='skyblue')
            ax2.set_title('Seasonal Precipitation (inches)')
            
            # Add total precipitation
            if 'precip_total' in metrics:
                ax2.text(0.05, 0.9, f"Annual Total: {metrics['precip_total']:.1f} inches",
                         transform=ax2.transAxes, bbox=dict(facecolor='white', alpha=0.8))
        elif 'precip_total' in metrics:
            # Alternative if seasonal data isn't available
            ax2.bar(['Annual Total'], [metrics['precip_total']], color='skyblue')
            ax2.set_title('Annual Precipitation (inches)')
        
        # Comfort metrics (bottom left)
        ax3 = axes[1, 0]
        comfort_metrics = []
        comfort_values = []
        
        # Add available comfort metrics
        if 'percent_comfortable' in metrics:
            comfort_metrics.append('Comfortable')
            comfort_values.append(metrics['percent_comfortable'])
        
        if 'percent_high_humidity' in metrics:
            comfort_metrics.append('High Humidity')
            comfort_values.append(metrics['percent_high_humidity'])
        
        if 'humidity_avg' in metrics:
            comfort_metrics.append('Avg Humidity')
            comfort_values.append(metrics['humidity_avg'])
        
        if comfort_metrics:
            # Create horizontal bar chart
            y_pos = range(len(comfort_metrics))
            ax3.barh(y_pos, comfort_values, color=['green', 'orange', 'blue'][:len(comfort_metrics)])
            ax3.set_yticks(y_pos)
            ax3.set_yticklabels(comfort_metrics)
            ax3.set_title('Comfort Metrics (% of time)')
            ax3.set_xlim(0, 100)  # Percentage scale
            
            # Add labels to the bars
            for i, v in enumerate(comfort_values):
                ax3.text(v + 1, i, f"{v:.1f}%", va='center')
        
        # Risk assessment (bottom right)
        ax4 = axes[1, 1]
        ax4.axis('off')  # No axes for this text panel
        
        # Set risk level with color
        risk_colors = {'high': 'red', 'medium': 'orange', 'low': 'green'}
        risk_color = risk_colors.get(analysis['risk_level'], 'gray')
        
        # Display risk level
        ax4.text(0.5, 0.9, f"WEATHER RISK: {analysis['risk_level'].upper()}", 
                 ha='center', fontsize=16, fontweight='bold', color=risk_color,
                 transform=ax4.transAxes)
        
        # Display risk score
        ax4.text(0.5, 0.8, f"Risk Score: {analysis['risk_score']:.1f}/100", 
                 ha='center', fontsize=14, transform=ax4.transAxes)
        
        # List risk factors
        if analysis['risk_factors']:
            ax4.text(0.5, 0.7, "Risk Factors:", ha='center', fontsize=12, 
                     fontweight='bold', transform=ax4.transAxes)
            
            for i, factor in enumerate(analysis['risk_factors'][:3]):  # Show top 3
                ax4.text(0.5, 0.65 - i*0.07, f"• {factor}", ha='center', fontsize=10,
                         transform=ax4.transAxes, wrap=True)
        
        # List positive factors
        if analysis['positive_factors']:
            pos_y = 0.7 - (len(analysis['risk_factors'][:3]) * 0.07) - 0.1
            ax4.text(0.5, pos_y, "Positive Factors:", ha='center', fontsize=12,
                     fontweight='bold', color='green', transform=ax4.transAxes)
            
            for i, factor in enumerate(analysis['positive_factors'][:2]):  # Show top 2
                ax4.text(0.5, pos_y - 0.07 - i*0.07, f"• {factor}", ha='center', 
                         fontsize=10, color='green', transform=ax4.transAxes, wrap=True)
        
        # Add timestamp
        fig.text(0.95, 0.01, f"Generated: {analysis['timestamp']}", 
                 ha='right', fontsize=8, color='gray')
        
        plt.tight_layout()
        plt.subplots_adjust(top=0.9)
        
        return fig

    def generate_weather_metrics_for_prediction(self, city, state):
        """
        Generate weather metrics specifically formatted for price prediction models
        
        Args:
            city: City name
            state: State code
            
        Returns:
            Dictionary with weather metrics suitable for prediction models
        """
        # Get full analysis
        analysis = self.analyze_weather_data(city, state)
        metrics = analysis['metrics']
        
        # Create a simplified set of metrics for prediction models
        prediction_metrics = {
            'weather_risk_score': analysis['risk_score'],
            'weather_risk_level': analysis['risk_level'],
        }
        
        # Add key metrics that would be useful for prediction
        if 'temp_avg' in metrics:
            prediction_metrics['avg_temperature'] = metrics['temp_avg']
        
        if 'precip_total' in metrics:
            prediction_metrics['annual_precipitation'] = metrics['precip_total']
        
        if 'days_over_90' in metrics:
            prediction_metrics['extreme_heat_days'] = metrics['days_over_90']
        
        if 'days_below_32' in metrics:
            prediction_metrics['freezing_days'] = metrics['days_below_32']
        
        if 'percent_comfortable' in metrics:
            prediction_metrics['comfort_index'] = metrics['percent_comfortable']
        
        if 'heavy_rain_hours' in metrics:
            prediction_metrics['heavy_rain_frequency'] = metrics['heavy_rain_hours']
        
        # Add normalized risk scores (0-1 scale) for key factors
        # These can be directly used as features in prediction models
        prediction_metrics['temp_risk'] = min(1.0, metrics.get('days_over_90', 0) / 120 + metrics.get('days_below_32', 0) / 120)
        prediction_metrics['precip_risk'] = min(1.0, abs(metrics.get('precip_total', 30) - 30) / 30)
        prediction_metrics['comfort_risk'] = max(0.0, 1.0 - metrics.get('percent_comfortable', 50) / 100)
        
        return prediction_metrics


# Test functionality
if __name__ == "__main__":
    # Create analyzer
    analyzer = HourlyWeatherAnalyzer()
    
    # Set API token if available (not required for sample data)
    api_token = "HrLmaYzJYcXfoXZvFJiKvOzTTJLLvBDv"  # Replace with your actual token
    analyzer.set_api_token(api_token)
    
    # Test with a sample location
    test_city = "Austin"
    test_state = "TX"
    
    try:
        # Get hourly weather data (will use sample data if API fails)
        print(f"Analyzing weather data for {test_city}, {test_state}...")
        weather_analysis = analyzer.analyze_weather_data(test_city, test_state)
        
        # Print risk assessment
        print(f"\nWeather Risk Assessment: {weather_analysis['risk_level'].upper()}")
        print(f"Risk Score: {weather_analysis['risk_score']:.1f}/100")
        
        # Print risk factors
        print("\nRisk Factors:")
        for factor in weather_analysis['risk_factors']:
            print(f"- {factor}")
        
        # Print positive factors
        print("\nPositive Factors:")
        for factor in weather_analysis['positive_factors']:
            print(f"- {factor}")
        
        # Generate and print weather summary
        summary = analyzer.generate_weather_summary(test_city, test_state)
        print("\n" + summary)
        
        # Create visualization
        fig = analyzer.plot_weather_risk(test_city, test_state)
        plt.savefig(f"{test_state}_{test_city}_weather_risk.png")
        plt.close(fig)
        print(f"\nSaved visualization to {test_state}_{test_city}_weather_risk.png")
        
        # Get prediction metrics
        prediction_metrics = analyzer.generate_weather_metrics_for_prediction(test_city, test_state)
        print("\nPrediction Metrics:")
        for key, value in prediction_metrics.items():
            print(f"- {key}: {value}")
        
    except Exception as e:
        print(f"Error in testing: {e}")

Analyzing weather data for Austin, TX...
Fetching hourly weather data for Austin, TX...
No data returned from API. Using generated sample data.
Generating sample hourly weather data for Austin, TX...
Generated 8761 hours of sample weather data

Weather Risk Assessment: LOW
Risk Score: 28.0/100

Risk Factors:
- High heat: 80.8 days above 90°F per year
- High precipitation: 50.3 inches annually
- Poor comfort: Only 19.7% of hours have ideal conditions

Positive Factors:
- Minimal freezing days
- Low humidity levels
Using cached hourly weather data for Austin, TX
Error in testing: Can only use .dt accessor with datetimelike values


/var/folders/2n/nsb_559s3znd30djr8w76s1w0000gn/T/ipykernel_12355/2598282420.py:384: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  season_temps = df.groupby('season')['temperature'].mean().to_dict()
/var/folders/2n/nsb_559s3znd30djr8w76s1w0000gn/T/ipykernel_12355/2598282420.py:388: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  season_precip = df.groupby('season')['precipitation'].sum().to_dict()


In [18]:
processor = DataProcessor()


In [19]:
merged = processor.load_merged_data("/Users/shivangi/Downloads/merged_fema_redfin_dataset.csv")
fema = processor.load_fema_data("/Users/shivangi/Downloads/fema_cleaned.csv")
summary = processor.get_state_disaster_summary("CALIFORNIA")
print(summary)


Loaded merged data: 5812149 rows, 57 columns
Loaded FEMA data: 20000 rows, 9 columns
{'total_disasters': 14672005.0, 'yearly_avg': 2445334.1666666665, 'risk_level': 'high'}


In [24]:
# Count records for ALABAMA
alabama_count = processor.merged_df[processor.merged_df['STATE'] == 'CONNECTICUT'].shape[0]
print(f"CONNECTICUT has {alabama_count} records in the merged dataset.")

CONNECTICUT has 69155 records in the merged dataset.


In [25]:
# Filter all records where STATE is CALIFORNIA
california_df = processor.merged_df[processor.merged_df['STATE'] == 'CONNECTICUT']

# Save to Excel
output_path = "/Users/shivangi/Downloads/CONNECTICUT_real_estate_data.xlsx"
california_df.to_excel(output_path, index=False)

print(f"Saved {len(california_df)} records for CONNECTICUT to:\n{output_path} ✅")


Saved 69155 records for CONNECTICUT to:
/Users/shivangi/Downloads/CONNECTICUT_real_estate_data.xlsx ✅


In [31]:
# Merge logic (same as you have)
merged_df = pd.read_csv("/Users/shivangi/Downloads/merged_fema_redfin_dataset.csv")
disaster_scores = pd.read_csv("/Users/shivangi/Downloads/City_Natural_Disaster_Score__0_30_realistic_.csv")

merged_df['CITY'] = merged_df['CITY'].astype(str).str.title().str.strip()
merged_df['STATE'] = merged_df['STATE'].astype(str).str.upper().str.strip()
disaster_scores['CITY'] = disaster_scores['CITY'].astype(str).str.title().str.strip()
disaster_scores['STATE'] = disaster_scores['STATE'].astype(str).str.upper().str.strip()

merged_result = pd.merge(merged_df, disaster_scores, on=['STATE', 'CITY'], how='left')

# ✅ Save to Downloads
merged_result.to_csv("/Users/shivangi/Downloads/redfin_with_disaster_score.csv", index=False)

print("✅ File saved to ~/Downloads/redfin_with_disaster_score.csv")


✅ File saved to ~/Downloads/redfin_with_disaster_score.csv


In [32]:
# Step 5: Get top 1000 rows
top_1000 = merged_result.head(1000)

# Step 6: Save to CSV in current directory
output_file = "redfin_with_disaster_score_top1000.csv"
top_1000.to_csv(output_file, index=False)

print(f"✅ Top 1000 rows with disaster scores saved to: {output_file}")

✅ Top 1000 rows with disaster scores saved to: redfin_with_disaster_score_top1000.csv


In [49]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm

# Load city disaster data
city_disaster = pd.read_csv('/Users/shivangi/Downloads/City_Natural_Disaster_Score__0_30_realistic_.csv')
city_disaster

,STATE,CITY,natural_disaster_score
0,ALABAMA,Abanda,22
1,ALABAMA,Abbeville,19
2,ALABAMA,Adamsville,20
3,ALABAMA,Akron,22
4,ALABAMA,Alabaster,18
...,...,...,...
22946,WYOMING,Wamsutter,1
22947,WYOMING,Washam,1
22948,WYOMING,Wilson,3
22949,WYOMING,Woods Landing-Jelm,5


In [55]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm

# Load city disaster data
city_disaster = pd.read_csv('/Users/shivangi/Downloads/City_Natural_Disaster_Score__0_30_realistic_.csv')
unique_cities = city_disaster[['STATE', 'CITY', 'natural_disaster_score']].drop_duplicates()

# Set date range: past three years, daily
start_date = datetime(2022, 5, 3)
end_date = datetime(2025, 5, 2)  # inclusive
total_days = (end_date - start_date).days + 1
timestamps = [start_date + timedelta(days=d) for d in range(total_days)]

def generate_weather(city, state, disaster_score):
    # Base values influenced by disaster score
    base_temp = 70 - disaster_score * 0.7
    temp_range = 20 + disaster_score * 0.5
    base_wind = 8 + disaster_score * 0.3
    base_precip = disaster_score * 0.08 * 24  # Convert hourly to daily total
    base_humidity = 60 + disaster_score * 0.6

    city_data = []
    for ts in timestamps:
        day_of_year = ts.timetuple().tm_yday
        # Seasonal variation
        season_adj = 10 * np.sin((2 * np.pi * (day_of_year - 172)) / 365.25)
        # Daily temp (average for the day)
        temp = base_temp + temp_range * 0.5 + season_adj + np.random.normal(0, 1.5)
        
        city_data.append({
            'state': state,
            'city': city,
            'date': ts.date(),
            'avg_temp': round(temp, 1),
            'min_temp': round(temp - np.random.uniform(2,4), 1),
            'max_temp': round(temp + np.random.uniform(2,4), 1),
            'wind_speed': round(base_wind + np.random.normal(0, 1.2), 1),
            'precipitation': round(max(0, base_precip + np.random.normal(0, 0.5)), 2),
            'humidity': round(base_humidity + np.random.normal(0, 5), 1),
            'pressure': round(1013 + np.random.normal(0, 2), 1),
            'disaster_score': disaster_score
        })
    return city_data

# Generate and save data in chunks
chunk_size = 5  # Adjust based on memory
all_city_indices = list(unique_cities.index)

for i in tqdm(range(0, len(all_city_indices), chunk_size)):
    chunk_indices = all_city_indices[i:i+chunk_size]
    weather_data = []
    for idx in chunk_indices:
        row = unique_cities.loc[idx]
        weather_data.extend(generate_weather(row['CITY'], row['STATE'], row['natural_disaster_score']))
    
    weather_df = pd.DataFrame(weather_data)
    if i == 0:
        weather_df.to_csv('synthetic_noaa_daily_precipitation.csv', index=False, mode='w')
    else:
        weather_df.to_csv('synthetic_noaa_daily_precipitation.csv', index=False, header=False, mode='a')

100%|███████████████████████████████████████| 4591/4591 [08:03<00:00,  9.49it/s]


In [57]:
ppt_disaster = pd.read_csv('/Users/shivangi/Desktop/Project/synthetic_noaa_daily_precipitation.csv')
ppt_disaster.head()

,state,city,date,avg_temp,min_temp,max_temp,wind_speed,precipitation,humidity,pressure,disaster_score
0,ALABAMA,Abanda,2022-05-03,60.9,58.5,63.0,14.0,42.39,78.5,1016.8,22
1,ALABAMA,Abanda,2022-05-04,64.6,62.3,67.5,15.5,42.81,78.6,1009.7,22
2,ALABAMA,Abanda,2022-05-05,61.7,57.7,64.8,13.9,41.49,75.5,1016.5,22
3,ALABAMA,Abanda,2022-05-06,65.9,63.4,69.3,13.6,41.81,71.0,1011.6,22
4,ALABAMA,Abanda,2022-05-07,61.5,59.1,65.1,14.6,42.62,78.6,1014.2,22


In [59]:
import pandas as pd

# Step 1: Load NOAA daily data
noaa_df = pd.read_csv("/Users/shivangi/Desktop/Project/synthetic_noaa_daily_precipitation.csv")
noaa_df['date'] = pd.to_datetime(noaa_df['date'], errors='coerce')
noaa_df['YEAR'] = noaa_df['date'].dt.year
noaa_df['MONTH'] = noaa_df['date'].dt.month
noaa_df['STATE'] = noaa_df['state'].str.upper().str.strip()
noaa_df['CITY'] = noaa_df['city'].str.title().str.strip()

# Step 2: Aggregate NOAA to monthly per city-state
noaa_monthly = noaa_df.groupby(['STATE', 'CITY', 'YEAR', 'MONTH']).agg({
    'avg_temp': 'mean',
    'min_temp': 'mean',
    'max_temp': 'mean',
    'wind_speed': 'mean',
    'precipitation': 'sum',
    'humidity': 'mean',
    'pressure': 'mean',
    'disaster_score': 'mean',
    'date': 'count'  # number of records contributing
}).reset_index().rename(columns={'date': 'source_count'})

print(f"NOAA monthly shape: {noaa_monthly.shape}")
print(noaa_monthly.head())

# Step 3: Load Redfin monthly data
redfin_df = pd.read_csv("redfin_with_disaster_score.csv")
redfin_df['PERIOD_BEGIN'] = pd.to_datetime(redfin_df['PERIOD_BEGIN'], errors='coerce')
redfin_df['YEAR'] = redfin_df['PERIOD_BEGIN'].dt.year
redfin_df['MONTH'] = redfin_df['PERIOD_BEGIN'].dt.month
redfin_df['STATE'] = redfin_df['STATE'].str.upper().str.strip()
redfin_df['CITY'] = redfin_df['CITY'].str.title().str.strip()

print(f"Redfin shape: {redfin_df.shape}")

# Step 4: Merge Redfin with NOAA monthly data
merged_df = pd.merge(
    redfin_df,
    noaa_monthly,
    on=['STATE', 'CITY', 'YEAR', 'MONTH'],
    how='left'
)

print("Merged data sample:")
print(merged_df[['CITY', 'STATE', 'YEAR', 'MONTH', 'MEDIAN_SALE_PRICE', 'avg_temp', 'disaster_score', 'source_count']].head())

# Step 5: Save output
merged_df.to_csv("/Users/shivangi/Downloads/redfin_with_fema_noaa.csv", index=False)
print("✅ Merged dataset saved to redfin_with_fema_noaa.csv")


NOAA monthly shape: (849187, 13)
     STATE    CITY  YEAR  MONTH   avg_temp   min_temp   max_temp  wind_speed  \
0  ALABAMA  Abanda  2022      5  64.324138  61.320690  67.362069   14.648276   
1  ALABAMA  Abanda  2022      6  69.410000  66.466667  72.516667   14.603333   
2  ALABAMA  Abanda  2022      7  74.535484  71.532258  77.441935   14.803226   
3  ALABAMA  Abanda  2022      8  77.819355  74.929032  80.832258   14.838710   
4  ALABAMA  Abanda  2022      9  80.203333  77.206667  83.086667   14.620000   

   precipitation   humidity     pressure  disaster_score  source_count  
0        1227.26  73.986207  1013.110345            22.0            29  
1        1264.88  74.396667  1013.733333            22.0            30  
2        1310.22  72.412903  1013.277419            22.0            31  
3        1307.60  74.219355  1013.570968            22.0            31  
4        1266.00  74.656667  1013.180000            22.0            30  
Redfin shape: (5812149, 59)
Merged data sample:


In [60]:
# Check percentage of rows with missing weather data
missing_weather = merged_df['avg_temp'].isna().sum()
total_rows = len(merged_df)
missing_pct = (missing_weather / total_rows) * 100

print(f"Total rows: {total_rows}")
print(f"Rows missing weather data: {missing_weather} ({missing_pct:.2f}%)")

# Check if missing data is evenly distributed across states/years
missing_by_state = merged_df[merged_df['avg_temp'].isna()].groupby('STATE').size()
missing_by_year = merged_df[merged_df['avg_temp'].isna()].groupby('YEAR').size()

print("\nMissing weather data by state (top 5):")
print(missing_by_state.sort_values(ascending=False).head())

print("\nMissing weather data by year:")
print(missing_by_year.sort_values())

Total rows: 5812149
Rows missing weather data: 4488408 (77.22%)

Missing weather data by state (top 5):
STATE
CALIFORNIA      365423
NEW YORK        334401
PENNSYLVANIA    311991
FLORIDA         282490
ILLINOIS        218891
dtype: int64

Missing weather data by year:
YEAR
2022    155064
2012    342681
2013    369510
2014    396069
2015    433551
2016    450540
2017    457520
2018    459413
2019    463513
2020    469833
2021    490714
dtype: int64


In [2]:
import pandas as pd
import numpy as np

def fill_weather_data_simple(merged_df):
    """
    Fill missing weather data with month-year medians.
    This uses one straightforward approach: fill all missing values for a given
    month and year with the median values from available data for that same month and year.
    
    Parameters:
    -----------
    merged_df : pandas DataFrame
        The merged Redfin and NOAA dataset with missing weather values
        
    Returns:
    --------
    pandas DataFrame
        The dataset with filled weather values
    """
    print("Starting simple weather data imputation...")
    
    # Create a copy to avoid modifying the original
    df = merged_df.copy()
    
    # Identify weather columns that need filling
    weather_cols = ['avg_temp', 'min_temp', 'max_temp', 'wind_speed', 
                   'precipitation', 'humidity', 'pressure', 'disaster_score']
    
    # Count missing values before filling
    missing_before = {col: df[col].isna().sum() for col in weather_cols}
    print(f"Missing values before: {missing_before}")
    
    # Calculate medians for each month-year combination
    print("Calculating month-year medians...")
    month_year_medians = df.groupby(['YEAR', 'MONTH'])[weather_cols].transform('median')
    
    # Fill missing values with these medians
    for col in weather_cols:
        df[col] = df[col].fillna(month_year_medians[col])
    
    # Count missing values after month-year filling
    missing_after_month_year = {col: df[col].isna().sum() for col in weather_cols}
    print(f"Missing values after month-year filling: {missing_after_month_year}")
    
    # If any values are still missing, fill with overall median
    if any(count > 0 for count in missing_after_month_year.values()):
        print("Some month-years have no data. Filling remaining missing values with overall medians...")
        for col in weather_cols:
            if df[col].isna().sum() > 0:
                overall_median = df[col].median()
                df[col] = df[col].fillna(overall_median)
    
    # Count final missing values
    missing_final = {col: df[col].isna().sum() for col in weather_cols}
    print(f"Final missing values: {missing_final}")
    
    # Add a flag to indicate which rows had their weather data filled
    df['weather_data_filled'] = False
    original_mask = merged_df['avg_temp'].notna()
    df.loc[original_mask, 'weather_data_filled'] = False
    df.loc[~original_mask, 'weather_data_filled'] = True
    
    filled_count = (~original_mask).sum()
    print(f"Filled weather data for {filled_count} rows ({filled_count/len(df)*100:.2f}% of dataset)")
    
    return df

# Example usage
if __name__ == "__main__":
    # Load the merged data
    merged_df = pd.read_csv("/Users/shivangi/Downloads/redfin_with_fema_noaa.csv")
    
    # Fill missing weather data
    filled_df = fill_weather_data_simple(merged_df)
    
    # Save the results
    filled_df.to_csv("/Users/shivangi/Downloads/filled_redfin_noaa_data.csv", index=False)
    
    print("Weather data filling complete!")

Starting simple weather data imputation...
Missing values before: {'avg_temp': 4488408, 'min_temp': 4488408, 'max_temp': 4488408, 'wind_speed': 4488408, 'precipitation': 4488408, 'humidity': 4488408, 'pressure': 4488408, 'disaster_score': 4488408}
Calculating month-year medians...
Missing values after month-year filling: {'avg_temp': 4488408, 'min_temp': 4488408, 'max_temp': 4488408, 'wind_speed': 4488408, 'precipitation': 4488408, 'humidity': 4488408, 'pressure': 4488408, 'disaster_score': 4488408}
Some month-years have no data. Filling remaining missing values with overall medians...
Final missing values: {'avg_temp': 0, 'min_temp': 0, 'max_temp': 0, 'wind_speed': 0, 'precipitation': 0, 'humidity': 0, 'pressure': 0, 'disaster_score': 0}
Filled weather data for 4488408 rows (77.22% of dataset)
Weather data filling complete!


In [9]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Load cleaned data
df = pd.read_csv("/Users/shivangi/Downloads/filled_redfin_noaa_data.csv")
df.head(50)



,PERIOD_BEGIN,PERIOD_END,PERIOD_DURATION,REGION_TYPE,IS_SEASONALLY_ADJUSTED,REGION,CITY,STATE,STATE_CODE,PROPERTY_TYPE,...,avg_temp,min_temp,max_temp,wind_speed,precipitation,humidity,pressure,disaster_score,source_count,weather_data_filled
0,2015-10-01,2015-10-31,30,place,False,"Indian Trail, NC",Indian Trail,NORTH CAROLINA,NC,Single Family Residential,...,74.026667,71.029032,77.030000,11.861290,750.20,67.812903,1013.000000,13.0,NaN,True
1,2014-04-01,2014-04-30,30,place,False,"Hainesville, IL",Hainesville,ILLINOIS,IL,Condo/Co-op,...,74.026667,71.029032,77.030000,11.861290,750.20,67.812903,1013.000000,13.0,NaN,True
2,2017-10-01,2017-10-31,30,place,False,"Maysville, NC",Maysville,NORTH CAROLINA,NC,All Residential,...,74.026667,71.029032,77.030000,11.861290,750.20,67.812903,1013.000000,13.0,NaN,True
3,2024-07-01,2024-07-31,30,place,False,"Chewelah, WA",Chewelah,WASHINGTON,WA,Single Family Residential,...,80.354839,77.393548,83.332258,10.232258,539.54,64.106452,1013.070968,9.0,31.0,False
4,2022-01-01,2022-01-31,30,place,False,"Lombard, IL",Lombard,ILLINOIS,IL,Single Family Residential,...,74.026667,71.029032,77.030000,11.861290,750.20,67.812903,1013.000000,13.0,NaN,True
5,2020-07-01,2020-07-31,30,place,False,Augusta-Richmond County consolidated governmen...,Augusta-Richmond County Consolidated Governmen...,GEORGIA,GA,Single Family Residential,...,74.026667,71.029032,77.030000,11.861290,750.20,67.812903,1013.000000,13.0,NaN,True
6,2020-02-01,2020-02-29,30,place,False,"Concord, NC",Concord,NORTH CAROLINA,NC,Single Family Residential,...,74.026667,71.029032,77.030000,11.861290,750.20,67.812903,1013.000000,13.0,NaN,True
7,2024-03-01,2024-03-31,30,place,False,"Eldersburg, MD",Eldersburg,MARYLAND,MD,Condo/Co-op,...,64.609677,61.400000,67.477419,11.461290,716.18,66.877419,1013.167742,12.0,31.0,False
8,2013-08-01,2013-08-31,30,place,False,"Laplace, LA",Laplace,LOUISIANA,LA,Single Family Residential,...,74.026667,71.029032,77.030000,11.861290,750.20,67.812903,1013.000000,13.0,NaN,True
9,2014-01-01,2014-01-31,30,place,False,"Egypt Lake-Leto, FL",Egypt Lake-Leto,FLORIDA,FL,All Residential,...,74.026667,71.029032,77.030000,11.861290,750.20,67.812903,1013.000000,13.0,NaN,True


In [33]:
import pandas as pd

# Load only the first 500 rows
df = pd.read_csv("/Users/shivangi/Downloads/filled_redfin_noaa_data.csv", nrows=500)

# Save to Excel
df.to_excel("/Users/shivangi/Downloads/preview_first_500_rows.xlsx", index=False)

print("Saved first 500 rows to preview_first_500_rows.xlsx")


Saved first 500 rows to preview_first_500_rows.xlsx


In [18]:
import pandas as pd
import numpy as np

# Optimized risk calculation parameters
RISK_WEIGHTS = {
    'fema_disaster_count': {
        'thresholds': [0, 2, 5, 10, 20],
        'scores':    [10, 20, 35, 50, 60]
    },
    'natural_disaster_score': {
        'thresholds': [0, 10, 20, 30, 40],
        'scores':    [15, 30, 45, 60, 70]
    },
    'precipitation': {
        'thresholds': [0, 2, 4, 6, 8],
        'scores':    [10, 20, 30, 40, 50]
    },
    'INVENTORY': {
        'thresholds': [0, 50, 100, 200, 300],
        'scores':    [5, 10, 15, 20, 25]
    }
}

def calculate_balanced_risk(row):
    """Calculate risk with compressed scaling and minimum baseline"""
    total = 0
    for feature, params in RISK_WEIGHTS.items():
        value = row[feature]
        for i, threshold in enumerate(params['thresholds']):
            if value < threshold:
                total += params['scores'][i-1] if i > 0 else 0
                break
        else:
            total += params['scores'][-1]
    
    # Apply square root compression and normalize
    compressed = np.sqrt(total)
    max_possible = np.sqrt(sum(v['scores'][-1] for v in RISK_WEIGHTS.values()))
    return min(95, (compressed / max_possible) * 100)  # Hard cap at 95

# Load data
df = pd.read_csv('/Users/shivangi/Downloads/filled_redfin_noaa_data.csv')

# Calculate scores
df['RISK_SCORE'] = df.apply(calculate_balanced_risk, axis=1)

# Add minimum score floor of 5
df['RISK_SCORE'] = df['RISK_SCORE'].clip(lower=5)

print(df[['CITY', 'STATE', 'RISK_SCORE']]
      .sort_values('RISK_SCORE', ascending=False)
      .head(10))

                     CITY       STATE  RISK_SCORE
492992          La Quinta  CALIFORNIA        95.0
1975099  Huntington Beach  CALIFORNIA        95.0
4048912         La Quinta  CALIFORNIA        95.0
582772          La Quinta  CALIFORNIA        95.0
2406476       Lake Forest  CALIFORNIA        95.0
2628450  Huntington Beach  CALIFORNIA        95.0
2680375        Santa Rosa  CALIFORNIA        95.0
5149211         Vacaville  CALIFORNIA        95.0
2877050  Huntington Beach  CALIFORNIA        95.0
3021086  Huntington Beach  CALIFORNIA        95.0


In [38]:
import pandas as pd

# Load the uploaded Excel file
weather_df = pd.read_csv("/Users/shivangi/Downloads/filled_redfin_noaa_data.csv")

# Define the WeatherAnalyzer class
class WeatherAnalyzer:
    def __init__(self, df):
        self.df = df.copy()
        self.df = self.df.dropna(subset=['CITY', 'STATE'])  # Ensure basic location info

    def get_city_weather_report(self, city_name, state_name=None):
        # Standardize input
        city_name = city_name.strip().title()
        if state_name:
            state_name = state_name.strip().upper()
            data = self.df[(self.df['CITY'] == city_name) & (self.df['STATE'] == state_name)]
        else:
            data = self.df[self.df['CITY'] == city_name]

        if data.empty:
            return pd.DataFrame({'value': [f"No data available for {city_name}" + (f", {state_name}" if state_name else "")]})

        # Compute summary stats for that city
        numeric_columns = data.select_dtypes(include='number').columns
        summary = data[numeric_columns].agg('mean').round(2)

        # Add sample count
        summary['data_points'] = len(data)
        return summary.to_frame(name='value')

# Initialize analyzer
analyzer = WeatherAnalyzer(weather_df)

# Example usage: report for Manchester, CT (change values as needed)
city_report = analyzer.get_city_weather_report("Huntington Beach", "CALIFORNIA")

print(city_report)


                                    value
PERIOD_DURATION                     30.00
MEDIAN_SALE_PRICE               899880.27
MEDIAN_SALE_PRICE_MOM                0.02
MEDIAN_SALE_PRICE_YOY                0.09
MEDIAN_LIST_PRICE               894038.03
MEDIAN_LIST_PRICE_MOM                0.01
MEDIAN_LIST_PRICE_YOY                0.07
MEDIAN_PPSF                        485.43
MEDIAN_PPSF_MOM                      0.02
MEDIAN_PPSF_YOY                      0.08
MEDIAN_LIST_PPSF                   522.69
MEDIAN_LIST_PPSF_MOM                 0.01
MEDIAN_LIST_PPSF_YOY                 0.07
HOMES_SOLD                          64.95
HOMES_SOLD_MOM                       0.11
HOMES_SOLD_YOY                       0.17
PENDING_SALES                       75.20
PENDING_SALES_MOM                    0.07
PENDING_SALES_YOY                    0.10
NEW_LISTINGS                        73.92
NEW_LISTINGS_MOM                     0.09
NEW_LISTINGS_YOY                     0.08
INVENTORY                         

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

class WeatherMetricsAnalyzer:
    def __init__(self, data_path="/Users/shivangi/Downloads/filled_redfin_noaa_data.csv"):
        """
        Initialize the Weather Metrics Analyzer with a merged dataset path.
        
        Parameters:
        -----------
        data_path : str
            Path to the merged FEMA-Redfin-Weather dataset
        """
        self.data_path = data_path
        self.data = None
        self.states = []
        self.cities_by_state = {}
        
        # If data path provided, load the data
        if data_path:
            self.load_data(data_path)
        else:
            print("No data path provided. Using sample data for demonstration.")
            self.generate_sample_data()
    
    def load_data(self, data_path):
        """Load the merged dataset from a CSV file."""
        try:
            print(f"Loading data from {data_path}...")
            self.data = pd.read_csv(data_path)
            print(f"Merged data: {self.data.shape[0]} rows, {self.data.shape[1]} columns")
            
            # Initialize states and cities
            self.extract_states_and_cities()
        except Exception as e:
            print(f"Error loading data: {e}")
            self.generate_sample_data()
    
    def generate_sample_data(self, rows=5000):
        """Generate sample data for demonstration purposes."""
        print(f"Generating sample data with {rows} rows...")
        
        # Define sample states and cities
        sample_states = ["ALABAMA", "ALASKA", "ARIZONA", "ARKANSAS", "CALIFORNIA", 
                         "COLORADO", "CONNECTICUT", "DELAWARE", "FLORIDA", "GEORGIA"]
        
        cities_by_state = {
            "CONNECTICUT": ["Ansonia", "Ball Pond", "Baltic", "Bantam", "Bethel", 
                           "Middletown", "Hartford", "Bristol", "Norwich", "Manchester"],
            "CALIFORNIA": ["Los Angeles", "San Francisco", "San Diego", "Sacramento", 
                          "Oakland", "San Jose", "Fresno", "Long Beach", "Bakersfield"],
            "FLORIDA": ["Miami", "Orlando", "Tampa", "Jacksonville", "Fort Lauderdale", 
                       "Tallahassee", "Naples", "Key West", "Pensacola", "Gainesville"]
        }
        
        # Add random cities for other states
        for state in sample_states:
            if state not in cities_by_state:
                cities_by_state[state] = [f"City_{i}" for i in range(1, 11)]
        
        # Create empty dataframe
        self.data = pd.DataFrame()
        
        # Generate random data
        periods = pd.date_range(start='2010-01-01', end='2023-12-31', freq='M')
        
        data_rows = []
        for _ in range(rows):
            state = random.choice(sample_states)
            city = random.choice(cities_by_state[state])
            period = random.choice(periods)
            
            # Generate realistic weather data based on region
            if state == "FLORIDA":
                avg_temp = np.random.normal(72, 5)  # Close to your avg_temp value
                min_temp = avg_temp - np.random.normal(3, 1)  # Similar to your spread
                max_temp = avg_temp + np.random.normal(3, 1)
                humidity = np.random.normal(70, 5)  # Close to your humidity value
                wind_speed = np.random.normal(13, 3)  # Similar to your wind_speed
                precipitation = np.random.normal(970, 200)  # Matching your precipitation scale
                pressure = np.random.normal(1013, 2)  # Matches your pressure value
            elif state == "CALIFORNIA":
                avg_temp = np.random.normal(65, 7)
                min_temp = avg_temp - np.random.normal(3, 1)
                max_temp = avg_temp + np.random.normal(3, 1)
                humidity = np.random.normal(60, 8)
                wind_speed = np.random.normal(10, 4)
                precipitation = np.random.normal(500, 300)
                pressure = np.random.normal(1012, 3)
            elif state == "CONNECTICUT":
                avg_temp = np.random.normal(55, 15)
                min_temp = avg_temp - np.random.normal(4, 2)
                max_temp = avg_temp + np.random.normal(4, 2)
                humidity = np.random.normal(65, 10)
                wind_speed = np.random.normal(14, 5)
                precipitation = np.random.normal(1200, 300)
                pressure = np.random.normal(1010, 5)
            else:
                avg_temp = np.random.normal(65, 10)
                min_temp = avg_temp - np.random.normal(3, 2)
                max_temp = avg_temp + np.random.normal(3, 2)
                humidity = np.random.normal(65, 10)
                wind_speed = np.random.normal(12, 4)
                precipitation = np.random.normal(800, 400)
                pressure = np.random.normal(1011, 4)
            
            # Generate FEMA disaster data
            natural_disaster_score = np.random.normal(30, 10)  # Matches your natural_disaster_score
            disaster_score = np.random.normal(17, 5)  # Close to your disaster_score
            
            data_rows.append({
                "PERIOD_BEGIN": period,
                "STATE": state,
                "CITY": city,
                "avg_temp": max(0, min(110, avg_temp)),
                "min_temp": max(0, min(110, min_temp)),
                "max_temp": max(0, min(110, max_temp)),
                "humidity": max(0, min(100, humidity)),
                "wind_speed": max(0, wind_speed),
                "precipitation": max(0, precipitation),
                "pressure": max(980, min(1040, pressure)),
                "natural_disaster_score": max(0, natural_disaster_score),
                "disaster_score": max(0, disaster_score),
                "YEAR": period.year,
                "MONTH": period.month
            })
        
        self.data = pd.DataFrame(data_rows)
        
        # Initialize states and cities
        self.extract_states_and_cities()
    
    def extract_states_and_cities(self):
        """Extract unique states and cities from the dataset."""
        if self.data is not None:
            self.states = sorted(self.data['STATE'].unique())
            print(f"Found {len(self.states)} available states")
            print(f"Sample states: {', '.join(self.states[:10])}")
            
            # Extract cities by state
            for state in self.states:
                state_cities = sorted(self.data[self.data['STATE'] == state]['CITY'].unique())
                self.cities_by_state[state] = state_cities
    
    def get_states(self):
        """Return list of available states."""
        return self.states
    
    def get_cities(self, state):
        """Return list of cities for a given state."""
        return self.cities_by_state.get(state, [])
    
    def analyze_state(self, state):
        """Analyze state-level metrics and provide summary."""
        if state not in self.states:
            print(f"State {state} not found in the dataset.")
            return None
        
        state_data = self.data[self.data['STATE'] == state]
        cities = self.get_cities(state)
        
        print(f"\nState {state} has {len(cities)} cities")
        
        # Count cities with few records
        city_counts = state_data['CITY'].value_counts()
        cities_with_one_record = sum(city_counts == 1)
        cities_with_few_records = sum(city_counts < 5)
        
        print(f"Cities with only 1 record: {cities_with_one_record}")
        print(f"Cities with fewer than 5 records: {cities_with_few_records}")
        
        # Top cities by record count
        print("\nTop cities by record count:")
        for city, count in city_counts.head(5).items():
            print(f"- {city}: {count} records")
        
        print(f"\nSample cities: {', '.join(cities[:5])}")
        
        # Disaster summary
        years = state_data['YEAR'].nunique()
        total_disasters = state_data['natural_disaster_score'].sum()
        yearly_avg = total_disasters / years if years > 0 else 0
        
        print(f"\nDisaster summary for {state}:")
        print(f"Total disasters: {int(total_disasters)}")
        print(f"Yearly average: {yearly_avg:.2f}")
        print(f"Years covered: {years}")
        
        # Determine risk level
        if yearly_avg < 10:
            risk = "low"
        elif yearly_avg < 100:
            risk = "medium" 
        else:
            risk = "high"
        
        print(f"Risk level: {risk}")
        
        return {
            "state": state,
            "cities": len(cities),
            "total_disasters": int(total_disasters),
            "yearly_avg_disasters": yearly_avg,
            "years_covered": years,
            "risk_level": risk
        }
    
    def analyze_city_weather(self, state, city):
        """Analyze weather metrics for a specific city."""
        if state not in self.states or city not in self.get_cities(state):
            print(f"City {city}, {state} not found in the dataset.")
            return None
        
        city_data = self.data[(self.data['STATE'] == state) & (self.data['CITY'] == city)]
        record_count = len(city_data)
        
        print(f"\nTesting with {city}, {state}")
        print(f"Found {record_count} records")
        
        # Years covered
        years = city_data['YEAR'].nunique()
        print(f"Years covered: {years}")
        
        # Weather metrics analysis
        weather_metrics = {
            'avg_temp': 'Average Temperature (°F)',
            'min_temp': 'Minimum Temperature (°F)',
            'max_temp': 'Maximum Temperature (°F)',
            'humidity': 'Humidity (%)',
            'wind_speed': 'Wind Speed (mph)',
            'precipitation': 'Precipitation (mm/year)',
            'pressure': 'Atmospheric Pressure (hPa)'
        }
        
        results = {}
        
        print("\nWeather Metrics:")
        for metric, label in weather_metrics.items():
            if metric in city_data.columns:
                avg = city_data[metric].mean()
                min_val = city_data[metric].min()
                max_val = city_data[metric].max()
                std = city_data[metric].std()
                
                results[metric] = {
                    'avg': avg,
                    'min': min_val,
                    'max': max_val,
                    'std': std
                }
                
                print(f"- {label}: {avg:.2f} (Range: {min_val:.2f}-{max_val:.2f}, Std: {std:.2f})")
        
        # Seasonal analysis if we have month data
        if 'MONTH' in city_data.columns and len(city_data) > 20:
            print("\nSeasonal Weather Patterns:")
            
            # Define seasons
            city_data['SEASON'] = city_data['MONTH'].apply(lambda m: 
                                                   'Winter' if m in [12, 1, 2] else
                                                   'Spring' if m in [3, 4, 5] else
                                                   'Summer' if m in [6, 7, 8] else
                                                   'Fall')
            
            seasonal_metrics = {}
            
            for metric in ['avg_temp', 'precipitation', 'humidity']:
                if metric in city_data.columns:
                    seasonal_data = city_data.groupby('SEASON')[metric].mean().to_dict()
                    seasonal_metrics[metric] = seasonal_data
                    
                    print(f"- {weather_metrics[metric]} by Season:")
                    for season, value in seasonal_data.items():
                        print(f"  • {season}: {value:.2f}")
            
            results['seasonal'] = seasonal_metrics
        
        # Disaster analysis
        if 'natural_disaster_score' in city_data.columns:
            total_disasters = city_data['natural_disaster_score'].sum()
            yearly_avg = total_disasters / years if years > 0 else 0
            
            # Calculate risk level
            if yearly_avg < 5:
                risk = "low"
            elif yearly_avg < 20:
                risk = "medium" 
            else:
                risk = "high"
            
            print("\nDisaster summary:")
            print(f"- Total disasters: {int(total_disasters)}")
            print(f"- Yearly average: {yearly_avg:.2f}")
            print(f"- Risk level: {risk}")
            
            if 'natural_disaster_score' in city_data.columns:
                avg_score = city_data['natural_disaster_score'].mean()
                print(f"- Natural disaster score: {avg_score:.2f}/100")
            
            if 'disaster_score' in city_data.columns:
                avg_score = city_data['disaster_score'].mean()
                print(f"- Overall disaster score: {avg_score:.2f}/100")
            
            results['disasters'] = {
                'total': int(total_disasters),
                'yearly_avg': yearly_avg,
                'risk_level': risk
            }
        
        # Weather classification
        if 'avg_temp' in results and 'precipitation' in results and 'humidity' in results:
            # Temperature classification
            avg_temp = results['avg_temp']['avg']
            if avg_temp > 75:
                temp_class = "Hot"
            elif avg_temp > 65:
                temp_class = "Warm"
            elif avg_temp > 50:
                temp_class = "Mild"
            else:
                temp_class = "Cool"
            
            # Humidity classification
            avg_humid = results['humidity']['avg']
            if avg_humid > 70:
                humid_class = "Humid"
            elif avg_humid > 50:
                humid_class = "Moderate"
            else:
                humid_class = "Dry"
            
            # Precipitation classification
            avg_precip = results['precipitation']['avg']
            if avg_precip > 1200:
                precip_class = "Very Wet"
            elif avg_precip > 800:
                precip_class = "Wet"
            elif avg_precip > 500:
                precip_class = "Moderate Rainfall"
            else:
                precip_class = "Dry"
            
            climate = f"{temp_class}, {humid_class}, {precip_class}"
            print(f"\nClimate Classification: {climate}")
            
            results['climate_class'] = climate
        
        # Generate weather profile summary
        profile = self.generate_weather_profile(results)
        print(f"\nWeather Profile Summary:")
        print(profile)
        
        results['profile'] = profile
        
        return results
    
    def generate_weather_profile(self, results):
        """Generate a descriptive summary of the weather profile."""
        profile = ""
        
        # Temperature profile
        if 'avg_temp' in results:
            avg = results['avg_temp']['avg']
            range_val = results['max_temp']['avg'] - results['min_temp']['avg']
            
            if avg > 75:
                profile += "Hot climate with "
            elif avg > 65:
                profile += "Warm climate with "
            elif avg > 50:
                profile += "Mild climate with "
            else:
                profile += "Cool climate with "
            
            if range_val > 25:
                profile += "extreme temperature variations. "
            elif range_val > 15:
                profile += "significant temperature variations. "
            else:
                profile += "moderate temperature variations. "
        
        # Precipitation and humidity
        if 'precipitation' in results and 'humidity' in results:
            precip = results['precipitation']['avg']
            humid = results['humidity']['avg']
            
            if precip > 1200 and humid > 70:
                profile += "Very wet and humid conditions. "
            elif precip > 800 and humid > 65:
                profile += "Wet and moderately humid conditions. "
            elif precip > 500:
                profile += "Moderate rainfall. "
            else:
                profile += "Relatively dry conditions. "
        
        # Wind conditions
        if 'wind_speed' in results:
            wind = results['wind_speed']['avg']
            
            if wind > 15:
                profile += "High average wind speeds. "
            elif wind > 10:
                profile += "Moderate wind conditions. "
            else:
                profile += "Generally calm winds. "
        
        # Seasonal variations
        if 'seasonal' in results and 'avg_temp' in results['seasonal']:
            seasons = results['seasonal']['avg_temp']
            max_season = max(seasons.items(), key=lambda x: x[1])[0]
            min_season = min(seasons.items(), key=lambda x: x[1])[0]
            
            if max_season == 'Summer' and seasons['Summer'] - seasons['Winter'] > 20:
                profile += f"Distinct seasons with hot summers and cold winters. "
            elif max_season == 'Summer' and seasons['Summer'] - seasons['Winter'] > 10:
                profile += f"Moderate seasonal variations. "
            else:
                profile += f"Relatively stable year-round temperatures. "
        
        # Disaster risk
        if 'disasters' in results:
            risk = results['disasters']['risk_level']
            
            if risk == 'high':
                profile += "High risk of natural disasters. "
            elif risk == 'medium':
                profile += "Moderate risk of natural disasters. "
            else:
                profile += "Low risk of natural disasters. "
        
        return profile
    
    def compare_cities(self, city_list):
        """Compare weather metrics across multiple cities."""
        print("\nComparing Weather Metrics Across Cities:")
        print("=" * 50)
        
        if len(city_list) < 2:
            print("Need at least 2 cities to compare.")
            return None
        
        # Collect data for each city
        city_data = {}
        for state, city in city_list:
            results = self.analyze_city_weather(state, city)
            if results:
                city_data[f"{city}, {state}"] = results
        
        if len(city_data) < 2:
            print("Not enough valid cities found for comparison.")
            return None
        
        # Compare key metrics
        metrics_to_compare = ['avg_temp', 'precipitation', 'humidity', 'wind_speed']
        
        print("\nKey Weather Metrics Comparison:")
        print("-" * 50)
        
        for metric in metrics_to_compare:
            metric_values = {}
            for city_name, data in city_data.items():
                if metric in data:
                    metric_values[city_name] = data[metric]['avg']
            
            if metric_values:
                print(f"\n{metric.replace('_', ' ').title()}:")
                for city, value in sorted(metric_values.items(), key=lambda x: x[1], reverse=True):
                    print(f"  {city}: {value:.2f}")
        
        # Compare disaster risk
        print("\nDisaster Risk Comparison:")
        print("-" * 50)
        
        risk_data = {}
        for city_name, data in city_data.items():
            if 'disasters' in data:
                risk_data[city_name] = {
                    'yearly_avg': data['disasters']['yearly_avg'],
                    'risk_level': data['disasters']['risk_level']
                }
        
        if risk_data:
            for city, risk in sorted(risk_data.items(), key=lambda x: x[1]['yearly_avg'], reverse=True):
                print(f"  {city}: {risk['yearly_avg']:.2f} disasters/year ({risk['risk_level'].upper()} risk)")
        
        # Optional: Generate climate similarity matrix
        if len(city_data) >= 3:
            print("\nClimate Similarity:")
            print("-" * 50)
            
            for city1 in city_data:
                for city2 in city_data:
                    if city1 != city2:
                        similarity = self.calculate_climate_similarity(city_data[city1], city_data[city2])
                        similarity_text = "Very Similar" if similarity > 0.8 else \
                                         "Similar" if similarity > 0.6 else \
                                         "Somewhat Similar" if similarity > 0.4 else \
                                         "Different"
                        print(f"  {city1} & {city2}: {similarity_text}")
        
        return city_data
    
    def calculate_climate_similarity(self, city1_data, city2_data):
        """Calculate a simple climate similarity score between two cities."""
        metrics = ['avg_temp', 'precipitation', 'humidity', 'wind_speed']
        similarities = []
        
        for metric in metrics:
            if metric in city1_data and metric in city2_data:
                val1 = city1_data[metric]['avg']
                val2 = city2_data[metric]['avg']
                
                # Normalize the difference based on typical ranges
                if metric == 'avg_temp':
                    max_diff = 50  # Typical max temperature difference
                elif metric == 'precipitation':
                    max_diff = 1500  # Typical max precipitation difference
                elif metric == 'humidity':
                    max_diff = 70  # Typical max humidity difference
                elif metric == 'wind_speed':
                    max_diff = 20  # Typical max wind speed difference
                else:
                    max_diff = abs(val1 - val2) * 2  # Fallback
                
                # Calculate similarity (1 = identical, 0 = maximally different)
                diff = abs(val1 - val2)
                similarity = max(0, 1 - (diff / max_diff))
                similarities.append(similarity)
        
        # Return average similarity if we have data
        if similarities:
            return sum(similarities) / len(similarities)
        return 0
    
    def run_demo(self, state=None, city=None):
        """Run a demonstration of the weather analyzer."""
        if not state:
            state = "CONNECTICUT" if "CONNECTICUT" in self.states else self.states[0]
        
        if not city:
            cities = self.get_cities(state)
            city = "Bantam" if "Bantam" in cities else cities[0] if cities else None
        
        if not city:
            print(f"No cities found for {state}. Using state-level analysis only.")
            self.analyze_state(state)
            return
        
        print(f"\nWeather Metrics Analysis for {city}, {state}")
        print("=" * 50)
        
        # Run the analysis
        results = self.analyze_city_weather(state, city)
        
        # Compare with other cities
        other_cities = []
        for s in self.states[:3]:  # Use first 3 states
            if s != state:
                cities = self.get_cities(s)
                if cities:
                    other_cities.append((s, cities[0]))
        
        if other_cities:
            compare_list = [(state, city)] + other_cities
            self.compare_cities(compare_list)

# Example usage
if __name__ == "__main__":
    # Create analyzer with sample data
    analyzer = WeatherMetricsAnalyzer()
    
    # Run demo with Connecticut and Bantam
    analyzer.run_demo("CONNECTICUT", "Bantam")

Loading data from /Users/shivangi/Downloads/filled_redfin_noaa_data.csv...
Merged data: 5812149 rows, 69 columns
Found 51 available states
Sample states: ALABAMA, ALASKA, ARIZONA, ARKANSAS, CALIFORNIA, COLORADO, COLUMBIA, CONNECTICUT, DELAWARE, FLORIDA

Weather Metrics Analysis for Bantam, CONNECTICUT

Testing with Bantam, CONNECTICUT
Found 205 records
Years covered: 14

Weather Metrics:
- Average Temperature (°F): 74.66 (Range: 65.20-84.74, Std: 3.18)
- Minimum Temperature (°F): 71.67 (Range: 62.19-81.83, Std: 3.19)
- Maximum Temperature (°F): 77.66 (Range: 68.33-87.68, Std: 3.18)
- Humidity (%): 67.60 (Range: 65.58-68.51, Std: 0.58)
- Wind Speed (mph): 11.76 (Range: 10.95-11.86, Std: 0.23)
- Precipitation (mm/year): 728.66 (Range: 633.78-750.20, Std: 42.17)
- Atmospheric Pressure (hPa): 1013.02 (Range: 1012.11-1013.85, Std: 0.21)

Seasonal Weather Patterns:
- Average Temperature (°F) by Season:
  • Fall: 76.51
  • Spring: 72.72
  • Summer: 75.22
  • Winter: 73.97
- Precipitation (mm/